In [ ]:
%pip install "transformers" "accelerate" "langchain" "einops"

In [ ]:
%pip install langchain
%pip install sentence_transformers
%pip install chromadb

In [3]:
pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.3/93.3 MB 13.2 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [8]:
!nvidia-smi


Sat Jul 15 11:19:32 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            Off  | 00000000:00:05.0 Off |                    0 |
| N/A   

In [9]:
import torch
device = torch.device(1)  # Choose the desired GPU, e.g., "cuda:0"
torch.cuda.set_device(device)

In [ ]:
import torch
from transformers import pipeline
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)
# load model pipeline
generate_text = pipeline(model="databricks/dolly-v2-3b", quantization_config=quantization_config,
                         trust_remote_code=True, device_map= "cuda", return_full_text=True)


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

A new version of the following files was downloaded from https://huggingface.co/databricks/dolly-v2-7b:
- instruct_pipeline.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [14]:
# Hugging Face model pipeline
hf_pipeline = HuggingFacePipeline(pipeline=generate_text)

In [7]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline

In [ ]:
# Text Embedding Model
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [9]:
# Vector Database configurations
persist_directory = "/kaggle/input/vectordb/AllMini_Chroma_Tik_500" # add path to specific vector database
CHROMA_SETTINGS = Settings(
        chroma_db_impl='duckdb+parquet',
        persist_directory=persist_directory,
        anonymized_telemetry=False
)

In [10]:
# Vector Database instance
db = Chroma(persist_directory=persist_directory, embedding_function=embeddings, client_settings=CHROMA_SETTINGS)

In [11]:
# Sample Run on VectorDB for context retrieval
query = "What are the placement statistics?"
docs = db.similarity_search_with_score(query)
context = docs[0][0].page_content
context[:]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

" campus Institutes by Companies looking to add bright young placement process include Endurance International talent to their workforce. A good number of students Group, Verse Innovation Private Limited (Dailyhunt), have been recruited with attractive packages by Media.net, Morgan Stanley, OpsHub Technologies leading Companies like Microsoft, Amazon, Zomato, Pvt. Ltd. and MAQ Software. Apart from the regular Goldman Sachs, LinkedIn, Sprinklr, Flock, OYO placement activities, the Placement Cell initiated the and Phillips. The 2018-19 placement season saw process of tying up a group of current students with recruiters from various domains of industries offering Alumni of the Insititute to mentor and prepare them job profiles like Software Engineer, R&D Engineer, for the campus placement process. The Placement Business Technology Analyst, Data Scientist, Site Cell also started with assessments and interview Reliability Engineer, ASIC Design Engineer and preparation for pre-final year stu

In [18]:
# Prompt Template 
from langchain import PromptTemplate, LLMChain
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question} Explain in points.
Helpful Answer:"""

# prompt chain
prompt_with_context = PromptTemplate(
input_variables=["question", "context"],
template= prompt_template)

In [19]:
# LLM custom Chain for question answering 
llm_context_chain = LLMChain(llm=hf_pipeline, prompt=prompt_with_context)

In [20]:
import time
while True:
        # Get the user query 
        query = input("\nEnter a query: ")
        if query == "exit":
            break
        if query.strip() == "":
            continue

        # Get the answer from the chain
        start = time.time()
        # similarity search from vectorDB to get context
        docs = db.similarity_search_with_score(query)
        
        # Retrieved context.
        context = docs[0][0].page_content
        
        #Prediction from LLM QA chain through question and user query.
        res = llm_context_chain.predict(question = query,context = context)
        end = time.time()

        # Print the result
        print("\n\n> Question:")
        print(query)
        print(f"\n> Answer (took {round(end - start, 2)} s.):")
        print(res)


Enter a query:  What is the B.Tech curriculum?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]



> Question:
What is the B.Tech curriculum?

> Answer (took 16.73 s.):

The BTech (ICT) curriculum is as below:
1. Compulsory foundation courses
2. Electives (technical, science, open)
3. Internships and BTech projects (BTP)

The BTech (ICT) curriculum is divided in to 3 categories, called "Foundation", "Electives" and "Projects".
Compulsory foundation courses are required to be taken by every student in the program. These are Telecom, Information Technology, Electronics, and Communication stream.
Electives are added to both technical and humanities and social science skills of the program. These are formed by a set of courses, which add to both the technical strength and humanities and social science skills of the program. These include, BTech project (BTP).
A student can get specialization in BTech by choosing multiple tracks, where the electives can be in the following four formats:
1. ICT - Courses which have components of computer systems and communication tracks
2. Technical - C


Enter a query:  What are the placement statistics?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]



> Question:
What are the placement statistics?

> Answer (took 15.46 s.):

The following are the placement statistics:
1. In the year 2018-19, around 197 students of the B.Tech course, signed up for campus placement.
2. Out of the 197 students, only 50 were successfully placed with different employers.
3. The average salary package was around Rs.10.16 lac per annum.
4. The highest salary package was around Rs.39.30 lac.
5. In total, there were 90 students who did not opt for campus placement.

Based on the above statistics, it can be inferred that, out of the 197 students of the B.Tech course, only around 101 students were successfully placed. The rest of the 96 students, who didn't opt for campus placement, could be mostly due to a combination of the below mentioned factors:
1. Poor communication from the company during the campus placement season.
2. High cost of the camp.
3. Lack of awareness about the benefits of campus placement.

Hence, it can be inferred that around 101 out of


Enter a query:  Name some cse professors.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]



> Question:
Name some cse professors.

> Answer (took 16.78 s.):

Srimanta Mandal - 7 points, Vinay Palaparthy - 7 points, Jayprakash T Lalchandani - 8 points, Archana Nigam - 9 points, Hardik Sailor - 7 points, Associate Professor - 10 points, Assistant Professor - 15 points, Professor - 25 points, Professor - 25 points, Professor - 35 points, Professor - 45 points, Professor - 55 points, Assistant Professor - 20 points, Assistant Professor - 20 points, Assistant Professor - 35 points, Assistant Professor - 30 points, Assistant Professor - 20 points, Assistant Professor - 15 points, Assistant Professor - 10 points, Assistant Professor - 5 points, Assistant Professor - 2 points, Assistant Professor - 1 point, Assistant Professor - 1 point, Assistant Professor - 1 point, Assistant Professor - 1 point, Assistant Professor - 1 point, Assistant Professor - 1 point, Assistant Professor - 1 point, Assistant Professor - 1 point, Assistant Professor - 1 point, Assistant Professor - 1 point, 


Enter a query:  exit
